### DESAFIO Nº1
Alumna: Lara Rosenberg

In [1]:
#LIBRERIAS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
import numpy as np

#### CARGA DE DATOS

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

#### CONSIGNA

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### EJERCICIO 1

In [3]:
#Vectorizamos los documentos con Count Vectorizer
cvect = CountVectorizer()
X_train = cvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

X_test = cvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

In [4]:
idx = [0,5,15,33,55]
for i in idx:# medimos la similaridad coseno con todos los documentos de train
    cossim = cosine_similarity(X_train[i], X_train)[0]
    # los 5 documentos más similares:
    mostsim = np.argsort(cossim)[::-1][1:6]
    print('documento original: ',i, newsgroups_train.target_names[y_train[i]])
    print('documentos similares: ')
    for j in mostsim:
        print(j, newsgroups_train.target_names[y_train[j]])
    print('----------------------------------------------')
    
    

documento original:  0 rec.autos
documentos similares: 
8013 rec.autos
5282 rec.autos
5553 rec.autos
2108 rec.motorcycles
3112 rec.autos
----------------------------------------------
documento original:  5 talk.politics.guns
documentos similares: 
4564 talk.politics.mideast
6894 talk.politics.guns
1191 talk.politics.misc
3415 alt.atheism
4988 sci.med
----------------------------------------------
documento original:  15 alt.atheism
documentos similares: 
2741 talk.politics.misc
1629 alt.atheism
3415 alt.atheism
6894 talk.politics.guns
3838 soc.religion.christian
----------------------------------------------
documento original:  33 talk.politics.mideast
documentos similares: 
6880 talk.politics.misc
8224 talk.politics.mideast
4881 talk.religion.misc
7777 talk.politics.mideast
4399 talk.politics.mideast
----------------------------------------------
documento original:  55 sci.crypt
documentos similares: 
3410 sci.crypt
5759 talk.politics.mideast
4988 sci.med
6635 talk.politics.misc
69

Como podemos ver, la similaridad del coseno está funcionando: notamos que siempre dentro del top 5 está encontrando al menos un documento de la misma categoria que el documento original. Ademas, en general, parece haber relacion entre las categorias que trae como top 5 (independientemente de que no sea exactamente la categoria original)

Revisamos cada caso a detalle

#### DOCUMENTO 1

In [5]:
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [6]:
newsgroups_train.data[8013]

"\nIf you don't already know it, you should call the bank/credit union/\nfinance company that holds the loan on your present car and get the\ncurrent payoff cost.\n\nIf you are trading in your current car on the new car, subtract the\npayoff amount from the trade-in the dealer is giving you.  (If this\nturns out to be a negative number, you need to reconsider the deal.)\nSubtract this difference from the price of the new car.  This is the\nsize of the loan you will need for the new car.\n\nThe dealer will take care of paying off the loan on your old car out\nof the money you give them when you pick up your new car.\n\nAt least that's how it worked for me 5 years ago in Ohio...\n\n\n\n"

In [7]:
newsgroups_train.data[5282]

"Alright, beat this automobile sighting.\n\nDriving along just a hair north of Atlanta, I noticed an old, run down\nformer car dealership which appeared to deal with, and repair, older\nrare or exotic foreign sports cars. I saw:\n\nFord GT-40 (!), the famous model from Ford, that seemed to win most of \nits races in the late 60s, including Le-Mans 4 or 6 times.\n\nTwo Jensen Interceptors, one a convertable, one a hatchback?\n\nPorsche 911 (boring compared to the rest)\n\nTHREE Ferarries, a Mondial, a 308 prepared for racing, and a red 60s model\nthat I couldn't identify.\n\nAnd at the bottom, a late 70s MG convertable.\n\nOutside there was a rotting Rover 3500 saloon, which was never regularly\nsold in the U.S.\n\nAnd in the showroom, there was a small italian body, either an Alpha Romeo\nor a Lancia. It was about the size of an Austin Mini.\nThe trunklid was missing, exposing a boot with a voltage regulator \nin the upper left corner of the wall, and a chunk of metal removed from\nthe

In [8]:
newsgroups_train.data[5553]

"Hi, maybe someone can help me here...\nI am looking to buy this 1990 Nissan Maxima GXE for CDN$14000 right now.\nThe car has 96000 km (or about 60000 miles) on it.\nA typical mileage for 1990 cars seem to be about 70000 km (or about 43K mi).\nThe seller just informed me that when he brought the car in for certification\nhe was told that the front break pads and the exhausts had to be replaced\nto meet the legal standards.  (He said he will replace the components before\nselling the car to me.)\n\nBeing copmletely ignorant to the technical stuff on cars, I don't know\nwhat this could mean...\nIs 96K km about the time typical for replacing the above mentioned items?\nOr is this an indication that the car was abused?\nWould other things break down or have to be replaced soon?\nThe seller told me that he used the car on the highway a lot, but,\nI don't know how to verify this...  I've seen the paint chipped away\nin tiny dots in the front edge of the hood, though.\n\nAlthough the Maxima i

In [9]:
newsgroups_train.data[2108]

'DS>From: viking@iastate.edu (Dan Sorenson)\n\n\nDS>>Riding up the hill leading to my\nDS>>house, I encountered a liver-and-white Springer Spaniel (no relation to\nDS>>the Springer Softail, or the Springer Spagthorpe, a close relation to\nDS>>the Spagthorpe Viking).\n\nDS>        I must have missed the article on the Spagthorpe Viking.  Was\nDS>that the one with the little illuminated Dragon\'s Head on the front\nDS>fender, a style later copied by Indian, and the round side covers?\n\nNo. Not at all. The Viking was a trick little unit made way back when\n(forties? fifties?) when Spag was trying to make a go of it in racing.\nThe first iteration (the Springer) was a boxer twin, very similar to Max\nFriz\'s famous design, but with an overhead "point cam" (see below for\nmore on the valvetrain). The problem was that the thing had no ground\nclearance whatsoever. The solution was to curve the cylinder bores, so\nthat the ground clearance was substantially increased:\n\n\n      ==@==   <-Sp

In [10]:
newsgroups_train.data[3112]

'...\n...\n\nSome other owners on the ford-probe@world.std.com mailing list have commented \nabout this problem, but I haven\'t seen it happen on my 3 month old \n\'93 Probe GT.  I think this may be something that Ford has corrected since \nthe initial batch of cars.  Also, someone said that Ford has issued a \nservice bulletin to inform dealers about how to correct this problem, so you \nmay be able to get your dealer to fix it.\n\n\nHaven\'t seen any of these problems, nor have I seen these mentioned on the\nmailing list, so these might just be having bad luck with these.\n\n\nDefinitely!\n\n\nNo problems like these either.  I haven\'t been able to find any obvious \nplaces where they screwed up.\n\n\nHmmm!  I got two keys with my car.  Something tells me you got stiffed by\nyour dealer on this.  (Actually, if you count the "credit card" key that \ncame in the owner\'s binder, I got 3 keys, but since the credit card key\ndoesn\'t have any teeth cut in it yet, I guess I won\'t count i

En este caso se puede notar que los textos son similares, ya que hablan de autos o motocicletas. La palabra auto se repite en casi todos los documentos y tambien aparecen palabras relacionadas a la marca de los autos (Yamaha, Ford, Alpha Romeo) o a las características de los autos.

#### DOCUMENTO 2

In [11]:
newsgroups_train.data[5]

'\n\n\n\n\nOf course.  The term must be rigidly defined in any bill.\n\n\nI doubt she uses this term for that.  You are using a quote allegedly\nfrom her, can you back it up?\n\n\n\n\nI read the article as presenting first an argument about weapons of mass\ndestruction (as commonly understood) and then switching to other topics.\nThe first point evidently was to show that not all weapons should be\nallowed, and then the later analysis was, given this understanding, to\nconsider another class.\n\n\n\n'

In [12]:
newsgroups_train.data[4564]

'\n[ stuff deleted ]\n   |> Are you calling names, or giving me a title? If the first, read your \n   |> paragraph above, if not I accept the title, in order to let you get into the\n   |> um, well, debate again.\n\n   Hasan replies:\n   I didnot know that "Master of wisdom" can be "name clling" too,\n   unless you consider yourself deserve-less !\n\nUnless you are referring to someone else, you have in fact given me a name \nI did not ask for, hence the term \'name calling\'.\n\n   |>    So what do you expect me to tell you to tell you, Master of Wsidom, \n   |> \t\t\t\t\t\t\t       ^^^\n   |> ------------------------------------------------------------------\n   I replied:\n   |> If you insist on giving me names/titles I did not ask for you could at\n   |> least spell them correctly. /sigh.\n\n   Hasan gloats:\n   That was only to confuse you! (ha ha ha hey )\n\nHell-bent on retarding into childhood, no? \n\n   |>when you are intentionally neglecting the MOST important fact that \n  

In [13]:
newsgroups_train.data[6894]

"Here is a press release from the White House.\n\n President Clinton's Remarks On Waco With Q/A\n To: National Desk\n Contact: White House Office of the Press Secretary, 202-456-2100\n\n   WASHINGTON, April 20 -- Following are remarks by President \nClinton in a question and answer session with the press:\n\n1:36 P.M. EDT\n\n     THE PRESIDENT:  On February the 28th, four federal\nagents were killed in the line of duty trying to enforce the law\nagainst the Branch Davidian compound, which had illegally stockpiled\nweaponry and ammunition, and placed innocent children at risk.\nBecause the BATF operation had failed to meet its objective, a 51-day\nstandoff ensued.\n\n     The Federal Bureau of Investigation then made every\nreasonable effort to bring this perilous situation to an end without\nbloodshed and further loss of life.  The Bureau's efforts were\nultimately unavailing because the individual with whom they were\ndealing, David Koresh, was dangerous, irrational, and probably\nins

In [14]:
newsgroups_train.data[1191]

"THE WHITE HOUSE\n\n                  Office of the Press Secretary\n                  (Vancouver, British Columbia)\n_________________________________________________________________\nFor Immediate Release                               April 4, 1993\n\n\t     \n                PRESS CONFERENCE BY THE PRESIDENT\n                        WITH RUSSIAN PRESS\n\t     \n                           Canada Place\n                   Vancouver, British Columbia\n\n\n\n2:46 P.M. PDT\n\n\t     \t  \n\t     Q\t  I had two questions for both Presidents, so you \ncould probably answer for Boris, too.  (Laughter.)\n\t     \n\t     THE PRESIDENT:  I'll give you my answer, then I'll \ngive you Yeltsin's answer.  (Laughter.)\n\t     \n\t     Q\t  The first is that this is the meeting of the \nPresidents, so the money that's being promised is government \nmoney, and naturally it's going to be distributed through the \ngovernment.  But you've indicated that three-quarters are going \nto be going to business

In [15]:
newsgroups_train.data[3415]

'\nTrue.  At first, the news media seemed entranced by all the new gizmos\nthe military was using, not to mention the taped video transmissions from\nthe missiles as they zeroed in on their targets.  But later, and especially\nafter the bunker full of civilians was hit, they changed their tone.  It\nseemed to me that they didn\'t have the stomach for the reality of war,\nthat innocent people really do die and are maimed in warfare.  It\'s like\nthey were only pro-Gulf-War as long as it was "nice and clean" (smart\nmissiles dropping in on military HQs), but not when pictures of dead,\ndying, and maimed civilians started cropping up.  What naive hypocrites!\n\n\n[ discussion about blanket-bombing and A-bombs deleted.]\n                            ^^^^^^^^^\n\nI should have said here "militarily justified".  It seems from your\ncomments below that you understood this as meaning "morally justified".\nI apologize.\n\n\nI have often wondered about this.  I\'ve always thought that the first b

In [16]:
newsgroups_train.data[4988]

'GREAT post Martin.  Very informative, well-balanced, and humanitarian\nwithout neglecting the need for scientific rigor.\n\n\n(Cross-posted to alt.psychology.personality since some personality typing\nwill be discussed at the beginning - Note: I\'ve set all followups to sci.med\nsince most of my comments are more sci.med oriented and I\'m sure most of the\nreplies, if any, will be med-related.)\n\n\n\n\nThey are just responding in their natural way:  Hyper-Choleric Syndrome (HCS).\nOops, that is not a recognized "illness" in the psychological community,\nbetter not say that since it therefore must not, and never will, exist.  :^)\n\nActually, it is fascinating that a disproportionate number of physicians\nwill type out as NT (for those not familiar with the Myers-Briggs system,\njust e-mail me and I\'ll send a summary file to you).  In the general\npopulation, NT\'s comprise only about 12% of the population, but among\nphysicians it is much much higher (I don\'t know the exact percent

El documento 1 se centra en armas de destruccion masiva y su regularizacion. En la mayoria de los textos se repiten palabras como "derechos", "conflicto", "armas", "legitimidad", cuestiones de regulacion, etc, que parecen unir a la mayoria de los textos aunque el tópico que traten sea diferente.

#### DOCUMENTO 3

In [17]:
newsgroups_train.data[15]

'\nDon\'t be so sure.  Look what happened to Japanese citizens in the US during\nWorld War II.  If you\'re prepared to say "Let\'s round these people up and\nstick them in a concentration camp without trial", it\'s only a short step to\ngassing them without trial.  After all, it seems that the Nazis originally\nonly intended to imprison the Jews; the Final Solution was dreamt up partly\nbecause they couldn\'t afford to run the camps because of the devastation\ncaused by Goering\'s Total War.  Those who weren\'t gassed generally died of\nmalnutrition or disease.\n'

In [18]:
newsgroups_train.data[2741]

'\n       Horrors, appealing to popular principles.  Can we perhaps as the\nquestion of whether the Constitution might have been written to appeal\nto the principles, rather than, as you appear to believe, it was written\nwith something else in mind and "propoganda" put out by its supporters.\n\n       But let\'s be honest about something, here.  When was the last time\nyou brought up all the valid points against your own arguments?\n\n       Or are they simply propogranda?  We can\'t know what Phill *really*\nmeans because he\'s obviously using arguments designed to convince.\n\n\n       If they\'re true or false, regardles of why they were said, why\non earth did you make a point of calling them "propogranda?"  That\nwould seem to be irrelevent.\n\n\n       Oh, I have no argument with questioning them.  I don\'t believe\nthey no longer apply, but that\'s because I think most of them were\ngood arguments.  I\'m not entirely happy about the situation, because\nthey were obviously only 

In [19]:
newsgroups_train.data[1629]

'\nSize of armies, duration, numbers of casualties both absolute and as a\npercentage of those involved, geographical area and numbers of countries\ntoo, are all measures of size.  In this case I\'d say the relevant\nstatistic would be the number of combatants (total troops) compared to\ntotal casualties from among the total civilian population in the\naffected geographical area.\n\n\nVietnam and Korea might make good comparisons.\n\n\nWestern news in general, but in particular the American "mass media":\nCBS, NBC, ABC, etc.  The general tone of the news during the whole\nwar was one of "those poor, poor Iraqis" along with "look how precisely\nthis cruise missile blew this building to bits".\n\n\nI agree.\n\n\nPerhaps so.  And maybe the atomic bomb was a mistake too.  But that\'s easy\nto say from our "enlightened" viewpoint here in the 90\'s, right?  Back\nthen, it was *all-out* war, and Germany and Japan had to be squashed.\nAfter all, a million or more British had already died, hund

In [20]:
newsgroups_train.data[3415]

'\nTrue.  At first, the news media seemed entranced by all the new gizmos\nthe military was using, not to mention the taped video transmissions from\nthe missiles as they zeroed in on their targets.  But later, and especially\nafter the bunker full of civilians was hit, they changed their tone.  It\nseemed to me that they didn\'t have the stomach for the reality of war,\nthat innocent people really do die and are maimed in warfare.  It\'s like\nthey were only pro-Gulf-War as long as it was "nice and clean" (smart\nmissiles dropping in on military HQs), but not when pictures of dead,\ndying, and maimed civilians started cropping up.  What naive hypocrites!\n\n\n[ discussion about blanket-bombing and A-bombs deleted.]\n                            ^^^^^^^^^\n\nI should have said here "militarily justified".  It seems from your\ncomments below that you understood this as meaning "morally justified".\nI apologize.\n\n\nI have often wondered about this.  I\'ve always thought that the first b

In [21]:
newsgroups_train.data[6894]

"Here is a press release from the White House.\n\n President Clinton's Remarks On Waco With Q/A\n To: National Desk\n Contact: White House Office of the Press Secretary, 202-456-2100\n\n   WASHINGTON, April 20 -- Following are remarks by President \nClinton in a question and answer session with the press:\n\n1:36 P.M. EDT\n\n     THE PRESIDENT:  On February the 28th, four federal\nagents were killed in the line of duty trying to enforce the law\nagainst the Branch Davidian compound, which had illegally stockpiled\nweaponry and ammunition, and placed innocent children at risk.\nBecause the BATF operation had failed to meet its objective, a 51-day\nstandoff ensued.\n\n     The Federal Bureau of Investigation then made every\nreasonable effort to bring this perilous situation to an end without\nbloodshed and further loss of life.  The Bureau's efforts were\nultimately unavailing because the individual with whom they were\ndealing, David Koresh, was dangerous, irrational, and probably\nins

In [22]:
newsgroups_train.data[3838]

'Larry L. Overacker writes, responding to Simon:\n\n   I may be interesting to see some brief selections posted to the\n   net.  My understanding is that SSPX does not consider ITSELF in\n   schism or legitimately excommunicated.  But that\'s really beside\n   the point.  What does the Roman Catholic church say?\n   Excommunication can be real apart from formal excommunication, as\n   provided for in canon law.\n\nHere\'s some of the theology involved for the interested.\n\nThere is confusion over this issue of the SSPX\'s "schism"; often the\nbasic problem is lack of an ability to distinguish between:\n\n- true obedience\n- false obedience\n- disobedience\n- schism\n\nTake the various classifications of obedience first.  There are 2\nimportant elements involved here for my purposes:\n\n1) a command\n2) the response made to the command\n\nAs far as the command goes, commands can be LEGITIMATE, such as the\nPope ordering Catholics to not eat meat on Fridays.  Or they can be\nILLEGITIMAT

El documento 1 trata sobre la segunda guerra mundial y los textos relacionados hablan sobre la constitucion, militares, muertes, lo que es legítimo o no, etc, que son todos temas relacionados.

#### DOCUMENTO 4

In [24]:
newsgroups_train.data[33]

"\nHow do you define war?  Do seiges and constant attacks on villiages\ncount as acts of war, or is that only when the Jews do them?\nJanuary, 1948: Arab Liberation Army attacks Kfar Szold\n               1000 men attack Kfar Etzion, 14 miles south of Jerusalem,\n                    after cutting off the supply lines to it.\nAttacks on Yehiam (Western Galilee) and kibbutz Tirat Tzvi.\nBy Mid-March, The Jewish settlements in the Negev had been cut off from\n      land links with the rest of the Jewish population.\n         The Etzion group of villiages, near Hebron, had been cut off,\n            while 42 members of a convoy trying to supply Yehiam were\n            slaughtered, cutting off the villiage.\nJerusalem was under seige, being cut off from its supply route from\n     Tel Aviv (the bombed out supply trucks have been left on the side\n     of that road to this day in memoriam).  By this time, 1200 Jews \n     had been killed.\n\nOf course, this isn't war, since it's only the Ar

In [25]:
newsgroups_train.data[6880]

'SECRET PURPOSE OF FALKLANDS WAR;  [with IN-VISIBILITY Technology]\n\nDr. Beter AUDIO LETTER #74 of 80\n\nDigitized by Jon Volkoff, mail address eidetics@cerf.net\n\n"AUDIO LETTER(R)" is a registered trademark of Audio Books,\nInc., a Texas corporation, which originally produced this tape\nrecording.  Reproduced under open license granted by Audio\nBooks, Inc.\n\n-----------------------------------------------------------------\n\nThis is the Dr. Beter AUDIO LETTER(R), 1629 K St. NW, Washington,\nDC  20006.\n\n   Hello, my friends, this is Dr. Beter.  Today is April 30,\n1982, and this is my AUDIO LETTER(R) No. 74.\n\n  It\'s now been almost one month since war broke out in the South\nAtlantic.  On the surface it seems that it\'s only a dispute\nbetween Argentina and Great Britain over the barren, wind-swept\nFalkland Islands and South Georgia Island.  In reality, it\'s far\nmore than that.\n\n   The so-called Falklands crisis is just the visible tip of a\ngiant military operation.  Du

In [26]:
newsgroups_train.data[8224]

'From: Center for Policy Research <cpr>\nSubject: Symbiotics: Zionism-Antisemitism\n\n\nZionism and the Holocaust\n-------------------------- by Haim Bresheeth\n\nThe first point to note regarding the appropriation of the history\nof the Holocaust by Zionist propaganda is that Zionism without\nanti-semitism is impossible. Zionism agrees with the basic tenet\nof anti-Semitism, namely that Jews cannot live with non- Jews.\n\nThe history and roots of the Holocaust go back a long way. While\nthe industru of death and destruction did not operate before 1942,\nits roots were firmly placed in the 19th Century. Jewish\naspirations for emancipation emerged out of the national struggles\nin Europe. When the hopes for liberation through\nbourgeois-democratic change were dashed, other alternatives for\nimproving the lot of the Jews of Europe achieved prominence.\n\nThe socialist Bund, a mass movement with enormous following, had\nto contend with opposition from a new and small, almost\ninsignifica

In [27]:
newsgroups_train.data[4881]

'\nRef: Encyclopedia of Religion, Mircea Eliade\n\nMAGI: \n\n[Sneak Preview: Later still, eschatology and apocalyptics were a fertile meeting\n ground for Iranian and Judeo-Christian religions, as can be seen in the famous\n _Oracles of Hystaspes_, a work whose Iranian roots are undeniable and which\n most likely dates from the beginning of the Christian era, probably the\n second century CE (Widengren, 1968). The Zoroastrian doctrine of the Savior of\n the Future (Saoshyant) was the basis for the story of the coming of the Magi\n to Bethlehem in the _Gospel of Matthew_ (2:1-12).]\n\n  The Old Persian word magu, rendered in Greek by magos, is of uncertain \netymology. It may originally have meant "member of the tribe," as in the\nAvestan compound mogu-tbish ("hostile to a member of the tribe"). This meaning\nwould have been further resticted, among the Medes, to "member of the priestly\ntribe" and perhaps to "priest" (Benveniste, 1938; Boyce, 1982). The term is\nprobably of Median orig

In [28]:
newsgroups_train.data[7777]

"\n\nI know it doesn't make sense, but since when is 'Napoleon' about\nsense, anyway? Further striking bigoted and racist attitude of \ncertain Greeks still exists in our day. Most Greeks insist even \ntoday, that the 537 year-old capital of the Ottoman Empire should \nbe called not by its rightful name of Istanbul, but by its half \na millennium-old moniker 'Cons*(whatever).'\n\nEveryone knows that New York City was once called 'New Amsterdam'\nbut Dutch people do not persist on calling it that today. The name \nof Stalingrad too is long gone, replaced by Volgagrad. China's\nPeking traded its name for Beiging long ago. Ciudad Trujillo\nof the Dominican Republic is now Santa Domingo. Zimbabve's\nold colonial capital Salisburry became Harrare. These changes\nhave all been accepted officially by everyone in the world.\n\nBut, Greeks are still determined on calling the Turkish Istanbul \nby the name of 'Cons*.'\n\nHow can one explain this total intransigence? What makes Greeks\nso differe

In [29]:
newsgroups_train.data[4399]

'\nDA] Armenia is involved in fighting with Azarbaijan.  It is Armenian\nDA] soldiers from mainland Armenia that are shelling towns in Azarbaijan.\n\n\nAre you related to \'Arromdian\' of ASALA/SDPA/ARF Terrorism and Revisionism\nTriangle? If you feel that you can simply act as a fascist Armenian \ngovernmental crony in this forum you will be sadly mistaken and duly \nembarrassed. This is not a lecture to another historical revisionist \nand a genocide apologist, but a fact. This time, fascist x-Soviet Armenian \nGovernment will not get away with the genocide of 204,000 Azeri men, women \nand children. Not a chance.\n\n The SUNDAY TIMES 8 March 1992\n Morgues fill as Azeris head for all-out war\n -------------------------------------------\n Thomas Goltz, the first to report the massacre by Armenian soldiers in\n the worst violence since the breakup of the Soviet Union, reports from\n Agdam\n ------\n Khojaly used to be a barren town, with empty shops and treeless dirt\n roads. Yet it 

El primer documento habla sobre conflictos en medio oriente. El segundo documento si bien no trata directamente la misma temática, habla de la guerra de Malvinas, que tambien es un conflicto territorial. La mayoria de los documentos hablan sobre guerra e independencia, tópicos altamente relacionados con los del conflicto de medio oriente.

#### DOCUMENTO 5

In [30]:
newsgroups_train.data[55]

"I sent a response to the White House at\n\n\t0005895485@MCIMAIL.COM (White House)\n\nand received a nice, automatic reply from MICMAIL noting, in passing, that\nif I had included a SNail address, I would get a reply in due course.\n\nFor those who care, my reply was:\n\n\t1.\tyes, let's protect the voice network\n\n\t2.\tprivately-developed crypto has always been available and\n\t\talways will be -- so let's think about how to do law\n\t\tenforcement given that fact not about how to hope to\n\t\tlegislate against it\n\n\t3.\tmy needs for crypto as a system designer are not met by the\n\t\tClipper Chip.  I want freely to export uses of algorithms\n\t\t(like DES & RSA) which are already freely available in the\n\t\tdestination country\n"

In [31]:
newsgroups_train.data[3410]

'Archive-name: net-privacy/part3\nLast-modified: 1993/3/3\nVersion: 2.1\n\n\nNOTES on ANONYMITY on the INTERNET\n==================================\n\nCompiled by L. Detweiler <ld231782@longs.lance.colostate.edu>.\n\n\n<8.1> What are some known anonymous remailing and posting sites?\n<8.2> What are the responsibilities associated with anonymity?\n<8.3> How do I `kill\' anonymous postings?\n<8.4> What is the history behind anonymous posting servers?\n<8.5> What is the value of anonymity?\n<8.6> Should anonymous posting to all groups be allowed?\n<8.7> What should system operators do with anonymous postings?\n<8.8> What is going on with anon.penet.fi maintained by J. Helsingius?\n\n\n* * *\n\n_____\n<8.1> What are some known anonymous remailing and posting sites?\n\n  Currently the most stable of anonymous remailing and posting sites\n  is anon.penet.fi operated by julf@penet.fi for several months, who\n  has system adminstrator privileges and owns the equipment. \n  Including anonymized

In [32]:
newsgroups_train.data[5759]

"From: Center for Policy Research <cpr>\nSubject: rejoinder. Questions to Israelis\n\n\nDear Josh\n\nI appreciate the fact that you sought to answer my questions.\n\nHaving said that, I am not totally happy with your answers.\n\n1.   You did not fully answer my question whether Israeli ID cards\nidentify the holders as Jews or Arabs. You imply that U.S.\ncitizens must identify themselves by RACE. Is that true ? Or are\njust trying to mislead the reader ? Do you know of any democratic\ncountry where people are asked to reveal their ethnical or\nreligious identity to any public official who so requests ?\n\n2.  The answer to the second question is evasive. There are all\nkinds of maps issued.  They are not equivalent to State policy.\nYou did not respond to my question.\n\n3.  Your answer to the third question (Israeli nuclear arsenal) is\ninteresting. You say that Israeli 'probably' stocks nuclear\nweapons. What evidence have you for maintaining that ?\n\n4.  My fourth question was answ

In [33]:
newsgroups_train.data[4988]

'GREAT post Martin.  Very informative, well-balanced, and humanitarian\nwithout neglecting the need for scientific rigor.\n\n\n(Cross-posted to alt.psychology.personality since some personality typing\nwill be discussed at the beginning - Note: I\'ve set all followups to sci.med\nsince most of my comments are more sci.med oriented and I\'m sure most of the\nreplies, if any, will be med-related.)\n\n\n\n\nThey are just responding in their natural way:  Hyper-Choleric Syndrome (HCS).\nOops, that is not a recognized "illness" in the psychological community,\nbetter not say that since it therefore must not, and never will, exist.  :^)\n\nActually, it is fascinating that a disproportionate number of physicians\nwill type out as NT (for those not familiar with the Myers-Briggs system,\njust e-mail me and I\'ll send a summary file to you).  In the general\npopulation, NT\'s comprise only about 12% of the population, but among\nphysicians it is much much higher (I don\'t know the exact percent

In [34]:
newsgroups_train.data[6635]

"THE WHITE HOUSE\n\n                    Office of the Press Secretary\n______________________________________________________________\nFor Immediate Release                             April 15, 1993     \n\n\t     \n                       REMARKS BY THE PRESIDENT\n                   TO LAW ENFORCEMENT ORGANIZATIONS\n\t     \n\t     \n                           The Rose Garden \n\n\n2:52 P.M. EDT\n\n\n\t     THE PRESIDENT:  Good afternoon.  Ladies and gentlemen, \ntwo months ago I presented a comprehensive plan to reduce our \nnational deficit and to increase our investment in the American \npeople, their jobs and their economic future.  The federal budget \nplan passed Congress in record time, and created a new sense of hope \nand opportunity in the country.  \n\t     \n\t     Then, the short-term jobs plan I presented to Congress, \nwhich would create a half a million jobs in the next two years passed \nthe House of Representatives two weeks ago.  It now has the support \nof a majori

In [35]:
newsgroups_train.data[6952]

'\nI hope I didn\'t award custody, Rich.  I purposely used "handle" in order to \navoid doing so - i.e., that happens to be what religions do (of course there are\naberrations like "scientific" creationism).  I used "best" in part to indicate \nthat science currently has a time of it with why and who, so these domains are\nmostly ignored.  I also attempted to be brief, which no doubt confused the\nmatter.  As an aside, for science I should have written "how and when".  Nobody\nseems to argue over what.\n\n\nSeems to me that the answer would vary from individual to individual.  I\'m not\ntrying to be evasive on this, but from a societal perspective, religion works.\nOn the other hand, sometimes it is abused and misused, and many suffer, which\nyou know.  But the net result seems positive, this from the anthropological\nperspective on human affairs.  You might call me a neo-Fruedian insofar as I \nthink the masses can\'t get along without religion.  Not that generally they are \nincapabl

El primer documento parece tratar sobre temas de comunicación gubernamental, criptografía, y preocupaciones sobre la legislación y la tecnología relacionada con la seguridad de la información. El segundo habla sobre posteos anonimos pero los demas textos no parecen estar altamente relacionados con el primer documento en su contenido.

### EJERCICIO 2

In [36]:
# Modelo 1: Naive Bayes multinomial sobre el CountVectorizer sin modificar ningun hiperparametro
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred =  mnb.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.5121480984265602

In [37]:
# Modelo 2: Naive Bayes multinomial sobre el CountVectorizer modificando fit_prior (determina si el modelo debe aprender las probabilidades a priori de cada clase a partir de los datos de entrenamiento. Si se establece en True, el modelo calculará las probabilidades de las clases basándose en su frecuencia relativa en el conjunto de entrenamiento. Si se establece en False, se utilizarán las probabilidades a priori uniformes (es decir, todas las clases tendrán la misma probabilidad a priori)).
mnb = MultinomialNB(fit_prior=False)
mnb.fit(X_train, y_train)
y_pred =  mnb.predict(X_test)
f1_score(y_test, y_pred, average='macro')


0.5183703099369554

In [38]:
# Modelo 3: Complement Naive Bayes sobre el CountVectorizer sin modificar ningun hiperparametro
cnb = ComplementNB()
cnb.fit(X_train, y_train)
y_pred =  cnb.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.637426504468489

In [39]:
# Modelo 4: Complement Naive Bayes sobre el CountVectorizer modificando el fit_prior
cnb = ComplementNB(fit_prior=False)
cnb.fit(X_train, y_train)
y_pred =  cnb.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.637426504468489

In [40]:
# Modelo 5: Complement Naive Bayes sobre el CountVectorizer modificando el norm (Si se establece en True, se normalizan los logaritmos de las probabilidades, lo que puede mejorar la estabilidad numérica del modelo. Esto es especialmente útil en conjuntos de datos desbalanceados)
cnb = ComplementNB(norm=True)
cnb.fit(X_train, y_train)
y_pred =  cnb.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6230316314296485

In [41]:
#Vectorizamos los documentos eliminando stopwords y evaluamos el mejor modelo que ya teniamos
cvect = CountVectorizer(stop_words='english')
X_train = cvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

X_test = cvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

In [42]:
mnb = MultinomialNB(fit_prior=False)
mnb.fit(X_train, y_train)
y_pred =  mnb.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6004045359469352

In [43]:
cnb = ComplementNB()
cnb.fit(X_train, y_train)
y_pred =  cnb.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6433458002117382

Podemos ver que las métricas de ambos modelos mejoran al eliminar stopwords, sin embargo, las métricas del Naive Bayes Multinomial mejoran casi 9 puntos respecto a la version con stopwords, mientras que en el Complementario, parece tener menos impacto hacer este preprocesamiento

In [44]:
# Utilizando un TF-IDF
tfidfvect = TfidfVectorizer()

X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

In [45]:
mnb = MultinomialNB(fit_prior=False)
mnb.fit(X_train, y_train)
y_pred =  mnb.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6013416381174239

In [46]:
cnb = ComplementNB()
cnb.fit(X_train, y_train)
y_pred =  cnb.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.692953349950875

Utilizando una Vectorizacion de TF-IDF, las métricas del Naive Bayes Complementario mejoran sustancialmente (suben casi 5 puntos), mientras que en el Multinomial casi no hay impacto

In [47]:
# Utilizando un TF-IDF con eliminacion de stopwords
tfidfvect = TfidfVectorizer(stop_words='english')

X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

In [48]:
mnb = MultinomialNB(fit_prior=False)
mnb.fit(X_train, y_train)
y_pred =  mnb.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6616846473821304

In [49]:
cnb = ComplementNB()
cnb.fit(X_train, y_train)
y_pred =  cnb.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6936107849650025

Conclusion: el mejor modelo es utilizando un Complement Naive Bayes y utilizando un TF-IDF

### EJERCICIO 3

In [50]:
#Vectorizamos los documentos
cvect = CountVectorizer()
X_train = cvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

X_test = cvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

In [51]:
# Transponer la matriz documento-término
X_train_t = X_train.transpose()

# Obtener el vocabulario
vocabulario = cvect.get_feature_names_out()

# Palabras a estudiar
palabras_a_estudiar = ['key', 'car', 'science', 'economics', 'gun']

# Obtener los índices de las palabras en el vocabulario
indices = [np.where(vocabulario == palabra)[0][0] for palabra in palabras_a_estudiar]

# Calcular la similaridad coseno
for idx in indices:
    cossim = cosine_similarity(X_train_t[idx], X_train_t)[0]
    # Obtener los 5 términos más similares
    mostsim = np.argsort(cossim)[::-1][1:6]
    
    print(f'Palabra original: {palabras_a_estudiar[indices.index(idx)]}')
    print('Palabras similares:')
    for j in mostsim:
        print(vocabulario[j])
    print('----------------------------------------------')

Palabra original: key
Palabras similares:
keys
encrypt
decrypt
encrypting
encrypted
----------------------------------------------
Palabra original: car
Palabras similares:
cars
tires
abs
brake
rotors
----------------------------------------------
Palabra original: science
Palabras similares:
scientific
cognitivists
behaviorists
psychology
methodology
----------------------------------------------
Palabra original: economics
Palabras similares:
ormation
harwood
ather
peasant
paisano
----------------------------------------------
Palabra original: gun
Palabras similares:
handgun
houk
licensee
blackman
danto
----------------------------------------------


- En el caso de la palabra key (llave), vemos que trae como palabras similares keys(llaves), encrypt (encriptar), decrypt(desencriptar). Ambas acciones suelen necesitar la key, con lo cual tiene sentido.
- Respecto de car (auto), nos trae como palabras similares cars (autos), brake (freno), tires(llantas) que son todas palabras que se encuentran fuertemente relacionadas con el auto.
- Para la palabra science (ciencia) nos devuelve palabras que estan sumamente relacionadas como metodología (que está muy arraigada a los procesos cientificos), psicologia, que es una ciencia, cientifico. 
- Respecto a la palabra economics (economia), nos devuelve palabras que no estan dentro de las que esperaba que devuelva.
- En el caso de gun (pistola), nos devuelve palabras como handgun (pistola) o licensee (licencia) que tienen relacion con la palabra original. Blackman o Danto, no me parecen palabras intuitivas para relacionar con gun.